In [1]:
import os, json
from pprint import pprint

In [64]:
with open('verbs_unparsed.json') as f:
    chunks = json.load(f)

In [65]:
for ch in chunks:
    ch['stem'] = list(map(lambda x: x.strip('.,;*()[]:&?! '), ch['stem'].split('|')))
    ch['lex'] = ch['lex'].strip('.,;*()[]:&?! ')

In [4]:
verbs = []
undefined = []
twos = []
undef_twos = []
for i in range(len(chunks)):
    if chunks[i]['gramm'] == 'V':
        if len(chunks[i]['stem'])<2:
            if '/' not in ''.join(chunks[i]['stem']):
                verbs.append(i)
            else:
                undefined.append(i)
        else:
            if '/' in ''.join(chunks[i]['stem']):
                undef_twos.append(i)
            else:
                twos.append(i)


len(verbs)

1716

In [5]:
print(len(twos))
print(len(undef_twos))
print(len(undefined))

156
179
64


In [66]:
for i in undefined:
    chunks[i]['stem'] = list(map(lambda x: x.strip('.'), chunks[i]['stem'][0].split('//')))


for i in undef_twos:
    chunks[i]['stem'] = [chunks[i]['stem'][0]] + chunks[i]['stem'][1].split('.//.')
    if '//' in ''.join(chunks[i]['stem']):
        print(i)
        pprint(chunks[i])

601
{'gramm': 'V',
 'lex': 'кылтыкыл',
 'paradigms': [],
 'stem': ['кылтыкыл//.кылт', 'кылтыт'],
 'trans_ru': '1) связка; 2) воз'}
1036
{'gramm': 'V',
 'lex': 'паннатъеқ',
 'paradigms': ['V-tr', 'V-intr'],
 'stem': ['паннатъеқ.//.паннэе', 'паннатъеқыт'],
 'trans_ru': '* топорок (птица)'}
1443
{'gramm': 'V',
 'lex': 'рымыркувык',
 'paradigms': [],
 'stem': ['рымыркув.//.рымыркэв', 'нмыркув', 'нмырков'],
 'trans_ru': 'облегчить (ношу)'}


In [67]:
for i in [601, 1036, 1443]:
    chunks[i]['stem'] = list(map(lambda x: x.strip('.'), chunks[i]['stem'][0].split('//'))) + chunks[i]['stem'][1:]

In [68]:
for i in range(len(chunks)):
    chunks[i]['lex'] = chunks[i]['lex'].replace('ң', 'ӈ')
    chunks[i]['lex'] = chunks[i]['lex'].replace('қ', 'ӄ')
    chunks[i]['lex'] = chunks[i]['lex'].replace('л', 'ԓ')
    chunks[i]['lex'] = chunks[i]['lex'].replace("'", 'ʼ')
    chunks[i]['stem'] = list(map(lambda x: x.replace('ң', 'ӈ'), chunks[i]['stem']))
    chunks[i]['stem'] = list(map(lambda x: x.replace('қ', 'ӄ'), chunks[i]['stem']))
    chunks[i]['stem'] = list(map(lambda x: x.replace('л', 'ԓ'), chunks[i]['stem']))
    chunks[i]['stem'] = list(map(lambda x: x.replace("'", 'ʼ'), chunks[i]['stem']))
    chunks[i]['stem'] = list(map(lambda x: x[:-1] if x[-1]=='ы' else x, chunks[i]['stem']))

In [10]:
folder = 'lexicons'
if not os.path.exists(os.path.join('..', folder)):
    os.mkdir(os.path.join('..', folder))

dict_file = 'verbs.lexc'
with open(os.path.join('..', folder, dict_file), 'w') as f:
#     f.write("""!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# !!!                          L E X I C O N                                  !!!
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# """)
    f.write("""
LEXICON Verbs 

""")

In [11]:
to_write = []
for i in verbs:
    x = chunks[i]
    to_write.append('{}:{} V ;    ! {}'.format(x['lex'], x['stem'][0], x['trans_ru']))
to_write.append('\n')

In [12]:
with open(os.path.join('..', folder, dict_file), 'a') as f:
    f.write('\n'.join(to_write))

In [69]:
to_write = []
still_not = []
nr = set('нр')
tn = set('тн')
for i in twos+undefined+undef_twos:
    x = chunks[i]

    if len(x['stem'][0])-len(x['stem'][1])==1 and x['stem'][0][1]=='ы':
        x['stem'] = list(map(lambda l: l[0]+'%{ы%}'+l[1:] if l[1]!='ы' else l[0]+'%{ы%}'+l[2:], x['stem']))
    if set(map(lambda y: y[1], x['stem'])) == set('ыъ'):
        x['stem'] = list(map(lambda l: l[0]+'%{ы%}'+l[2:], x['stem']))
    if set(map(lambda y: y[0], x['stem'])) == nr:
        x['stem'] = list(map(lambda l: '%{R%}'+l[1:], x['stem']))
    if set(map(lambda y: y[0], x['stem'])) == tn:
        x['stem'] = list(map(lambda l: 'т'+l[1:], x['stem']))
            
    if len(set(x['stem'])) == 1:
        to_write.append('{}:{} V ;    ! {}'.format(x['lex'], x['stem'][0], x['trans_ru']))
    else:
        still_not.append(x)
print(len(to_write))
len(still_not)

114


285

In [75]:
for x in still_not:
    if 'ык' in list(map(lambda y: y[-2:], x['stem'])):
        x['stem'] = list(map(lambda y: y.strip('ык'), x['stem']))

In [82]:
for x in still_not:
    for end in 'ԓьэт тку ԓьат тко ыԓьэт ытку ыԓьат ытко ръу ръо ыръу ыръо'.split():
        if x['stem'][1]==x['stem'][0]+end:
            x['stem']

no
no
no
no
no
no
no


In [76]:
for i in still_not[50:70]:
    pprint(i)
    print(len(set(i['stem'])))

{'gramm': 'V',
 'lex': 'гыпик',
 'paradigms': ['V-tr', 'V-intr'],
 'stem': ['гыпи', 'гыпиԓьэт'],
 'trans_ru': '1) заниматься домашними делами; 2) работать дома'}
2
{'gramm': 'V',
 'lex': 'ивыпэтык',
 'paradigms': ['V-tr', 'V-intr'],
 'stem': ['ивыпэт', 'игыпэт'],
 'trans_ru': 'стесняться'}
2
{'gramm': 'V',
 'lex': 'иткэк',
 'paradigms': ['V-tr', 'V-intr'],
 'stem': ['иткэ', 'иткэтку'],
 'trans_ru': 'отнимать'}
2
{'gramm': 'V',
 'lex': 'итчувйиӈэтык',
 'paradigms': ['V-tr', 'V-intr'],
 'stem': ['итчувйиӈэт', 'ичувчиӈэԓ'],
 'trans_ru': 'вздохнуть'}
2
{'gramm': 'V',
 'lex': 'йиԓюк',
 'paradigms': ['V-tr', 'V-intr'],
 'stem': ['йиԓю', 'йиԓютку'],
 'trans_ru': '1) заступаться; 2) защищать'}
2
{'gramm': 'V',
 'lex': 'йынатык',
 'paradigms': ['V-tr', 'V-intr'],
 'stem': ['йынат', 'йынатытко'],
 'trans_ru': 'просить, требовать (что-л.)'}
2
{'gramm': 'V',
 'lex': 'кагнотык',
 'paradigms': ['V-tr', 'V-intr'],
 'stem': ['агнот', 'агноткынат'],
 'trans_ru': 'вставать на цыпочки'}
2
{'gramm': 'V',


In [16]:
to_write = []
for i in twos+undefined+undef_twos:
    x = chunks[i]
    for i in range(len(x['stem'])):
        to_write.append('{}:{} V ;    ! {}'.format(x['lex'], x['stem'][i], x['trans_ru']))

In [17]:
with open(os.path.join('..', folder, 'verbu.lexc'), 'a') as f:
    f.write('\n'.join(to_write))